# ELECTRA for Question Answering on SQUAD
Trong notebook này ta sẽ làm quen với mô hình Electra ứng dụng cho bài toán Question Answering. Electra là một phương pháp học biểu diễn ngôn ngữ (language  representation learning) được ứng dụng cho nhiều bài toán khác nhau, ví dụ như Classification, QA, Text chunking. Đây là một phương pháp học biểu diễn mới, cho phép chúng ta đạt được hiệu năng cao với các Benchmark task trong NLP như SQUAD và GLUE (chi tiết xem tại paper [ ELECTRA: Pre-training Text Encoders as Discriminators Rather Than Generators](https://openreview.net/pdf?id=r1xMH1BtvB)).

Stanford Question Answering Dataset (SQuAD) là một dataset cho bài toán đọc hiểu và trả lời câu hỏi được phát triển bởi đại học Stanford. Trong đó, với mỗi bản ghi, một hệ thống AI sẽ được cung cấp một đoạn văn bản để đọc hiểu và một câu hỏi, nhiệm vụ của hệ thống AI đó là trả lời câu hỏi đó bằng một đoạn trích từ đoạn văn bản được cung cấp nếu có thể, hoặc báo lại là không thể trả lời nếu đoạn văn cung cấp không thể dùng để trả lời câu hỏi.

ELECTRA được công bố với ba phiên bản theo kích thước tăng dần như sau: Small, Base, Large. Vì giới hạn về thời gian cũng như khả năng tính toán, trong notebook này ta sẽ tiến hành thử nghiệm với mô hình ELECTRA Small. Học viên nên chạy bài thực hành này trên notebook nếu không có server để hỗ trợ


## Bước 1: Setup môi trường trên Google Colab
Học viên sử dụng nền tảng tính toán khác ngoài Google Colan có thể bỏ qua bước này. Trước khi chạy những câu lệnh dưới, ta chọn cấu hình GPU bằng cách ấn: **Runtime** -> **Change runtime type** -> **GPU**

### 1.1. Mount máy ảo vào drive của chúng ta

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


### 1.2. Cài đặt thư viện và tải mã nguồn ELECTRA

Trong bài thực hành này, ta sẽ sử dụng mã nguồn ELECTRA do bên Google Research phát triển. Để sử dụng mã nguồn này ta sẽ phải cài thư viện tensorflow==1.15 và

***Đầu tiên ta cài đặt tensorflow phiên bản 1.15***

In [2]:
!pip install tensorflow==1.15

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 412.3 MB 25 kB/s 
     |████████████████████████████████| 3.8 MB 52.8 MB/s 
     |████████████████████████████████| 503 kB 55.8 MB/s 
     |████████████████████████████████| 50 kB 7.9 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=e5b6f30e9a655cc983352889a1468c329ef7dd20a5f5e3baa2f0d4377deeb2a5
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting un

***Sau đó, ta clone git repo của ELECTRA về không gian làm việc và cd và thư mục `electra`***

In [3]:
!git clone https://github.com/google-research/electra.git

Cloning into 'electra'...
remote: Enumerating objects: 161, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 161 (delta 74), reused 65 (delta 65), pack-reused 64
Receiving objects: 100% (161/161), 113.72 KiB | 18.95 MiB/s, done.
Resolving deltas: 100% (87/87), done.


In [4]:
cd electra/

/content/electra


***Tiếp theo, ta download và unzip file mô hình của phiên bản ELECTRA Small***

In [5]:
!wget https://storage.googleapis.com/electra-data/electra_small.zip
!unzip electra_small.zip

--2022-06-22 02:52:18--  https://storage.googleapis.com/electra-data/electra_small.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.45.16, 172.217.0.48, 172.217.2.112, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.45.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 171877176 (164M) [application/zip]
Saving to: ‘electra_small.zip’

electra_small.zip   100%[===================>] 163.91M   150MB/s    in 1.1s    

2022-06-22 02:52:20 (150 MB/s) - ‘electra_small.zip’ saved [171877176/171877176]

Archive:  electra_small.zip
   creating: electra_small/
  inflating: electra_small/checkpoint  
  inflating: electra_small/electra_small.meta  
  inflating: electra_small/electra_small.data-00000-of-00001  
  inflating: electra_small/electra_small.index  
  inflating: electra_small/vocab.txt  


## Bước 2: Download và quan sát dữ liệu

### 2.1. Download training và validation data của bộ Squad 2.0

In [6]:
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

--2022-06-22 02:52:22--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘train-v2.0.json’

train-v2.0.json     100%[===================>]  40.17M   181MB/s    in 0.2s    

2022-06-22 02:52:22 (181 MB/s) - ‘train-v2.0.json’ saved [42123633/42123633]

--2022-06-22 02:52:22--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘dev-v2.0.json’

dev-v2.0.json       1

### 2.2. Tạo thư mục để chứa dữ liệu huấn luyện và chuyển data vào thư mục đó
***Đầu tiên, ta tạo một thư mục tên là `data` để chứa dữ liệu và file models***

*Trong đó, theo quy ước của mã nguồn:*
 - `finetuning_data/<tên tác vụ>` là thư mục chứa data cho tác vụ tương ứng, vì chúng ta đang làm bài toán Question Answering, nên tên thư mục con sẽ để là `squad`.
 - `models` là thư mục chứa model của electra mà ta muốn sử dụng

Sau khi tạo hai thư mục này rồi, ta chuyển hai file json chứa dữ liệu của SQuAD 2.0 vào thư mục `squad`

In [7]:
!mkdir -p data/finetuning_data/squad
!mkdir -p data/models/
!mv dev-v2.0.json data/finetuning_data/squad/dev.json
!mv train-v2.0.json data/finetuning_data/squad/train.json

Tiếp theo, ta copy file vocab.txt từ thư mục `electra_small` sang thư mục `data`

In [8]:
!cp electra_small/vocab.txt data/vocab.txt

Cuối cùng, ta copy thư mục `electra_small` vào trong thư mục `data/models`

In [9]:
import shutil
shutil.copytree('electra_small', 'data/models/electra_small', copy_function = shutil.copy) 

'data/models/electra_small'

### 2.3. Quan sát dữ liệu

Bây giờ, ta sẽ thực hiện một vài thao tác thống kê để hiểu thêm về dữ liệu của Squad

In [10]:
import os
os.listdir("data/finetuning_data/squad")

['train.json', 'dev.json']

In [11]:
import json
from pprint import pprint
import numpy as np


def view_squad_info(subset = 'train', get_impossible_exp = False):
    with open("data/finetuning_data/squad/{}.json".format(subset), "r") as f:
      data = json.load(f)
    
    # Thống kê số văn bản
    numOfParagraph = 0
    # YOUR CODE HERE
    for i in range(len(data["data"])):
      numOfParagraph += len(data["data"][0]['paragraphs'])
    # YOUR CODE HERE
    
    
    # Thống kê số cặp câu hỏi câu trả lời
    numOfQaPair = 0
     # YOUR CODE HERE
    for i in range(len(data["data"])):
      for j in range(len(data["data"][i]['paragraphs'])):
        numOfQaPair += len(data["data"][i]['paragraphs'][j]["qas"])
    # YOUR CODE HERE      

    
    # Thống kê độ dài của context
    ContextLen = []
    # YOUR CODE HERE
    for i in range(len(data["data"][0])):
      for j in range(len(data["data"][i]['paragraphs'])):
        ContextLen.append(len(data["data"][i]['paragraphs'][j]["context"]))
    maxContextLen = np.max(ContextLen)
    # YOUR CODE HERE
    

    # Thống kê độ dài của query và answer
    queryLen = [] # Độ dài của các query
    ansLen = [] # Độ dài của các câu trả lời

    # YOUR CODE HERE
    for i in range(len(data["data"])):
      for j in range(len(data["data"][i]['paragraphs'])):
        for k in range(len(data["data"][i]['paragraphs'][j]["qas"])):
          queryLen.append(len(data["data"][i]['paragraphs'][j]["qas"][k]["question"]))
          if len(data["data"][i]['paragraphs'][j]["qas"][k]["answers"]) > 0:
            ansLen.append(len(data["data"][i]['paragraphs'][j]["qas"][k]["answers"][0]["text"]))
    # YOUR CODE HERE


    avgQueLen = np.mean(queryLen)
    avgAnsLen = np.mean(ansLen)


    print("Phiên bản SQuAd là {}".format(data["version"]))
    print("Số văn bản trong dataset là {}".format(len(data["data"])))
    print("Mỗi văn bản có những key sau: {}".format(data["data"][0].keys()))
    print("Số đoạn văn trong dataset là: {}".format(numOfParagraph))
    print("Số cặp câu hỏi và trả lời trong dataset là: {}".format(numOfQaPair))
    print("Độ dài tối đa của một đoạn văn là: {}".format(maxContextLen))
    print("Độ dài trung bình của một câu hỏi là: {}".format(avgQueLen))
    print("Độ dài trung bình của một trả lời là: {}".format(avgAnsLen))
    
    print("------MỘT SỐ CẶP CÂU VÍ DỤ-----")
    pprint(data["data"][0]['paragraphs'][0]["qas"][0:2])
    print("-------------------------------")
    pprint(data["data"][-1]['paragraphs'][0]["qas"][0:2])

    if get_impossible_exp:
      for i in range(len(data["data"])):
        for j in range(len(data["data"][i]['paragraphs'])):
          for k in range(len(data["data"][i]['paragraphs'][j]["qas"])):
            if data["data"][i]['paragraphs'][j]["qas"][k]['is_impossible']:
              pprint(data["data"][i]['paragraphs'][j]["qas"][k])

Ta sử dụng hàm `view_squad_info` để xem thông tin của dataset:

In [12]:
view_squad_info(subset = 'train')

Phiên bản SQuAd là v2.0
Số văn bản trong dataset là 442
Mỗi văn bản có những key sau: dict_keys(['title', 'paragraphs'])
Số đoạn văn trong dataset là: 29172
Số cặp câu hỏi và trả lời trong dataset là: 130319
Độ dài tối đa của một đoạn văn là: 1895
Độ dài trung bình của một câu hỏi là: 58.50773870272178
Độ dài trung bình của một trả lời là: 20.149168979855105
------MỘT SỐ CẶP CÂU VÍ DỤ-----
[{'answers': [{'answer_start': 269, 'text': 'in the late 1990s'}],
  'id': '56be85543aeaaa14008c9063',
  'is_impossible': False,
  'question': 'When did Beyonce start becoming popular?'},
 {'answers': [{'answer_start': 207, 'text': 'singing and dancing'}],
  'id': '56be85543aeaaa14008c9065',
  'is_impossible': False,
  'question': 'What areas did Beyonce compete in when she was growing up?'}]
-------------------------------
[{'answers': [],
  'id': '5a7db48670df9f001a87505f',
  'is_impossible': True,
  'plausible_answers': [{'answer_start': 50,
                         'text': 'ordinary matter compos

In [13]:
view_squad_info(subset = 'dev', get_impossible_exp= False)

Phiên bản SQuAd là v2.0
Số văn bản trong dataset là 35
Mỗi văn bản có những key sau: dict_keys(['title', 'paragraphs'])
Số đoạn văn trong dataset là: 1365
Số cặp câu hỏi và trả lời trong dataset là: 11873
Độ dài tối đa của một đoạn văn là: 1765
Độ dài trung bình của một câu hỏi là: 59.50619051629748
Độ dài trung bình của một trả lời là: 20.916160593792174
------MỘT SỐ CẶP CÂU VÍ DỤ-----
[{'answers': [{'answer_start': 159, 'text': 'France'},
              {'answer_start': 159, 'text': 'France'},
              {'answer_start': 159, 'text': 'France'},
              {'answer_start': 159, 'text': 'France'}],
  'id': '56ddde6b9a695914005b9628',
  'is_impossible': False,
  'question': 'In what country is Normandy located?'},
 {'answers': [{'answer_start': 94, 'text': '10th and 11th centuries'},
              {'answer_start': 87, 'text': 'in the 10th and 11th centuries'},
              {'answer_start': 94, 'text': '10th and 11th centuries'},
              {'answer_start': 94, 'text': '10th and

Bên trên là một vài thông số cơ bản của SQuAD dataset. Nếu như ta muốn sử dụng lại mô hình ELECTRA cho bài toán Question-Answering, ta có thể làm hai việc sau:
- Xây dựng ngữ liệu cho ngôn ngữ mà bạn muốn xây dựng mô hình từ đó và tạo file vocab.txt tương ứng, sau đó chạy file run_finetuning.py trong mã nguồn để có được mô hình electra custom của bạn
- Thiết kế một dataset có cấu trúc giống như trên và lắp ghép với mã nguồn trong bài thực hành này.

Và đương nhiên, là phải có một server thật khỏe để chạy!

## Bước 3: Training

Ta chạy dòng lệnh dưới đây để thực hiện training, chúng ta có thể thay đổi các tham số trong hparams và theo dõi sự khác biệt trong quá trình huấn luyện

In [14]:
# YOUR CODE HERE
!python3 run_finetuning.py --data-dir data --model-name electra_small --hparams '{"model_size": "small", "task_names": ["squad"], "eval_batch_size": 16, "beam_size": 20, "train_batch_size": 32}'
# YOUR CODE HERE

Config: model=electra_small, trial 1/1
answerable_classifier True
answerable_uses_start_logits True
answerable_weight 0.5
beam_size 20
data_dir data
debug False
do_eval True
do_lower_case True
do_train True
doc_stride 128
double_unordered True
embedding_size 128
eval_batch_size 16
gcp_project None
init_checkpoint data/models/electra_small
iterations_per_loop 1000
joint_prediction True
keep_all_models True
layerwise_lr_decay 0.8
learning_rate 0.0001
log_examples False
max_answer_length 30
max_query_length 64
max_seq_length 512
model_dir data/models/electra_small/finetuning_models/squad_model
model_hparam_overrides {}
model_name electra_small
model_size small
n_best_size 20
n_writes_test 5
num_tpu_cores 1
num_train_epochs 2.0
num_trials 1
predict_batch_size 32
preprocessed_data_dir data/models/electra_small/finetuning_tfrecords/squad_tfrecords
qa_eval_file <built-in method format of str object at 0x7fe5520fdc00>
qa_na_file <built-in method format of str object at 0x7fe5520fdce0>
qa_na_th

In [15]:
# YOUR CODE HERE
with open("data/models/electra_small/results/squad_results.txt", "r") as f:
  result_file = f.read()

print(result_file.replace(" - ", "\n"))
# YOUR CODE HERE

squad: HasAns_exact: 37.92
HasAns_f1: 40.87
HasAns_total: 5928.00
NoAns_exact: 93.57
NoAns_f1: 93.57
NoAns_total: 5945.00
best_exact: 69.98
best_exact_thresh: -1.71
best_f1: 72.41
best_f1_thresh: -1.64
exact: 65.79
f1: 67.26
total: 11873.00



Mô hình đang đạt được độ chính xác tầm 65% và f1 là 67% trên tập test của Squad, ta có thể cải thiện mô hình bằng cách chỉnh các tham số cho phù hợp và chạy mô hình với số epoch lớn hơn. Nếu như bạn có hệ thống máy mạnh hơn, bạn có thể thử nghiệm với các phiên bản lớn hơn của electra

Sau khi chạy xong mô hình chúng ta có thể test bài toán bằng 1 pretrain model của transformer để hiểu rõ cũng như tối giản việc code. 

Tham khảo tài liệu ở [đây](https://huggingface.co/deepset/electra-base-squad2)

Dữ liệu test sẽ là:



```
'question': 'How does this work?',
'context' : 'This works fine.',
'context': 'This works perfectly.',
'context': 'Its not working at all.'
```
Hoặc bạn có thẻ thay đổi bất lỳ câu hỏi và trả lời bằng tiếng anh nào nếu muốn.


In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "deepset/electra-base-squad2"

# a) Get predictions
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
QA_input = {
    'question': 'How does this work?',
    'context' : 'This works fine.',
    'context': 'This works perfectly.',
    'context': 'Its not working at all.'
}
res = nlp(QA_input)

# b) Load model & tokenizer
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)